![pageindex_banner](https://pageindex.ai/static/images/pageindex_banner.jpg)

<p align="center"><i>Reasoning-based RAG&nbsp; ✧ &nbsp;No Vector DB&nbsp; ✧ &nbsp;No Chunking&nbsp; ✧ &nbsp;Human-like Retrieval</i></p>

<p align="center">
  <a href="https://vectify.ai">🏠 Homepage</a>&nbsp; • &nbsp;
  <a href="https://dash.pageindex.ai">🖥️ Dashboard</a>&nbsp; • &nbsp;
  <a href="https://docs.pageindex.ai/quickstart">📚 API Docs</a>&nbsp; • &nbsp;
  <a href="https://github.com/VectifyAI/PageIndex">📦 GitHub</a>&nbsp; • &nbsp;
  <a href="https://discord.com/invite/VuXuf29EUj">💬 Discord</a>&nbsp; • &nbsp;
  <a href="https://ii2abc2jejf.typeform.com/to/tK3AXl8T">✉️ Contact</a>&nbsp;
</p>

<div align="center">

[![Star us on GitHub](https://img.shields.io/github/stars/VectifyAI/PageIndex?style=for-the-badge&logo=github&label=⭐️%20Star%20Us)](https://github.com/VectifyAI/PageIndex) &nbsp;&nbsp; [![Follow us on X](https://img.shields.io/badge/Follow%20Us-000000?style=for-the-badge&logo=x&logoColor=white)](https://twitter.com/VectifyAI)

</div>

---

# Simple Vectorless RAG with PageIndex

## PageIndex Introduction
PageIndex is a new **reasoning-based**, **vectorless RAG** framework that performs retrieval in two steps:  
1. Generate a tree structure index of documents  
2. Perform reasoning-based retrieval through tree search  

<div align="center">
  <img src="https://docs.pageindex.ai/images/cookbook/vectorless-rag.png" width="70%">
</div>

Compared to traditional vector-based RAG, PageIndex features:
- **No Vectors Needed**: Uses document structure and LLM reasoning for retrieval.
- **No Chunking Needed**: Documents are organized into natural sections rather than artificial chunks.
- **Human-like Retrieval**: Simulates how human experts navigate and extract knowledge from complex documents. 
- **Transparent Retrieval Process**: Retrieval based on reasoning — say goodbye to approximate semantic search ("vibe retrieval").

## 📝 Notebook Overview

This notebook demonstrates a simple, minimal example of **vectorless RAG** with PageIndex. You will learn how to:
- [x] Build a PageIndex tree structure of a document
- [x] Perform reasoning-based retrieval with tree search
- [x] Generate answers based on the retrieved context

> ⚡ Note: This is a **minimal example** to illustrate PageIndex's core philosophy and idea, not its full capabilities. More advanced examples are coming soon.

---

## Step 0: Preparation



#### 0.1 Install PageIndex

In [ ]:
%pip install -q --upgrade pageindex

#### 0.2 Setup PageIndex

In [ ]:
import json
import os
from pageindex.utils import (
    print_toc, 
    remove_fields, 
    structure_to_list, 
    ChatGPT_API,
    print_json
)

# 配置模型 (使用本地/兼容 OpenAI 的 API)
MODEL = "qwen3-max"

# 封装 LLM 调用 (同步版本)
def call_llm(prompt, model=MODEL):
    """调用 LLM 生成回复"""
    return ChatGPT_API(model=model, prompt=prompt)

# 创建节点映射的辅助函数
def create_node_mapping(tree):
    """将树结构转换为 node_id -> node 的字典映射"""
    node_list = structure_to_list(tree)
    return {node['node_id']: node for node in node_list if 'node_id' in node}

print("本地 RAG 环境已配置完成")
print(f"使用模型: {MODEL}")

#### 0.3 配置环境变量

确保在 `.env` 文件中配置了以下环境变量:
- `CHATGPT_API_KEY`: 你的 API Key
- `BASE_URL`: API 基础地址 (默认为阿里云 DashScope)

In [ ]:
# 验证环境变量配置
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("CHATGPT_API_KEY")
base_url = os.getenv("BASE_URL", "https://dashscope.aliyuncs.com/compatible-mode/v1")

if api_key:
    print(f"API Key 已配置 (前8位): {api_key[:8]}...")
    print(f"Base URL: {base_url}")
else:
    print("警告: 未找到 CHATGPT_API_KEY 环境变量，请在 .env 文件中配置")

## Step 1: PageIndex Tree Generation

#### 1.1 加载本地生成的 PageIndex 树结构

使用本地已生成的结构文件，而非调用云端 API。

> 如需生成新的结构文件，请使用 `run_pageindex_md.py` 或 `page_index.py` 脚本。

In [ ]:
# 加载本地生成的结构文件
# 修改为你自己的结构文件路径
STRUCTURE_PATH = r"D:\github_dir\pageindex_project\results\汇总文档v1_structure.json"

# 检查文件是否存在
if not os.path.exists(STRUCTURE_PATH):
    print(f"错误: 结构文件不存在: {STRUCTURE_PATH}")
    print("请先使用 PageIndex 生成文档结构")
else:
    with open(STRUCTURE_PATH, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    tree = data['structure']
    print(f"成功加载结构文件: {STRUCTURE_PATH}")
    print(f"文档标题: {tree.get('title', '未知')}")

Downloaded https://arxiv.org/pdf/2501.12948.pdf
Document Submitted: pi-cmeseq08w00vt0bo3u6tr244g


#### 1.2 查看文档树结构

In [ ]:
# 打印文档目录结构
print("=== 文档目录结构 ===")
if isinstance(tree, dict):
    print_toc([tree])
elif isinstance(tree, list):
    print_toc(tree)

# 创建节点映射
node_map = create_node_mapping(tree)
print(f"\n=== 共 {len(node_map)} 个节点 ===")
for node_id, node in node_map.items():
    summary = node.get('summary', '无摘要')
    summary_preview = summary[:60] + '...' if len(summary) > 60 else summary
    print(f"  {node_id}: {node['title']} - {summary_preview}")

Simplified Tree Structure of the Document:
[{'title': 'DeepSeek-R1: Incentivizing Reasoning Cap...',
  'node_id': '0000',
  'prefix_summary': '# DeepSeek-R1: Incentivizing Reasoning C...',
  'nodes': [{'title': 'Abstract',
             'node_id': '0001',
             'summary': 'The partial document introduces two reas...'},
            {'title': 'Contents',
             'node_id': '0002',
             'summary': 'This partial document provides a detaile...'},
            {'title': '1. Introduction',
             'node_id': '0003',
             'prefix_summary': 'The partial document introduces recent a...',
             'nodes': [{'title': '1.1. Contributions',
                        'node_id': '0004',
                        'summary': 'This partial document outlines the main ...'},
                       {'title': '1.2. Summary of Evaluation Results',
                        'node_id': '0005',
                        'summary': 'The partial document provides a summary ...'}]},
    

## Step 2: Reasoning-Based Retrieval with Tree Search

#### 2.1 设置查询问题

In [ ]:
# 设置查询问题
query = "这份文档的主要内容是什么？"
print(f"查询问题: {query}")

#### 2.2 使用 LLM 进行树搜索，找出相关节点

In [ ]:
import re

# 对于只有一个节点的简单文档，直接使用该节点
if len(node_map) == 1:
    selected_node_ids = list(node_map.keys())
    print(f"文档只有一个节点，直接使用: {selected_node_ids[0]}")
else:
    # 多节点时使用 LLM 搜索
    # 构建节点摘要信息供 LLM 选择
    node_summaries = []
    for node_id, node in node_map.items():
        summary = node.get('summary', node.get('title', '无摘要'))
        node_summaries.append(f"- {node_id}: {node['title']} - {summary[:200]}")
    
    search_prompt = f"""你是一个文档检索助手。根据问题，从以下节点中找出最可能包含答案的节点。

问题: {query}

可用节点:
{chr(10).join(node_summaries)}

请只返回一个 JSON 数组，包含最相关的 node_id（最多3个），格式如: ["0001", "0003"]
不要返回其他内容。
"""
    
    print("正在调用 LLM 进行树搜索...")
    node_ids_response = call_llm(search_prompt)
    print(f"LLM 返回: {node_ids_response}")
    
    # 解析 LLM 返回的节点 ID
    try:
        match = re.search(r'\[.*?\]', node_ids_response, re.DOTALL)
        if match:
            parsed_ids = json.loads(match.group())
            # 过滤出有效的节点 ID
            selected_node_ids = [nid for nid in parsed_ids if nid in node_map]
            if not selected_node_ids:
                # 如果没有有效节点，使用第一个非根节点
                selected_node_ids = [list(node_map.keys())[1] if len(node_map) > 1 else list(node_map.keys())[0]]
                print(f"LLM 返回的节点无效，使用默认节点: {selected_node_ids}")
        else:
            selected_node_ids = [list(node_map.keys())[1] if len(node_map) > 1 else list(node_map.keys())[0]]
            print(f"无法解析 LLM 返回，使用默认节点: {selected_node_ids}")
    except Exception as e:
        print(f"解析错误: {e}")
        selected_node_ids = [list(node_map.keys())[1] if len(node_map) > 1 else list(node_map.keys())[0]]

print(f"\n选中的节点: {selected_node_ids}")
for nid in selected_node_ids:
    print(f"  - {nid}: {node_map[nid]['title']}")

Reasoning Process:
The question asks for the conclusions in the document. Typically, conclusions are found in sections
explicitly titled 'Conclusion' or in sections summarizing the findings and implications of the work.
In this document tree, node 0019 ('5. Conclusion, Limitations, and Future Work') is the most
directly relevant, as it is dedicated to the conclusion and related topics. Additionally, the
'Abstract' (node 0001) may contain a high-level summary that sometimes includes concluding remarks,
but it is less likely to contain the full conclusions. Other sections like 'Discussion' (node 0018)
may discuss implications but are not explicitly conclusions. Therefore, the primary node is 0019.

Retrieved Nodes:
Node ID: 0019	 Page: 16	 Title: 5. Conclusion, Limitations, and Future Work


## Step 3: Answer Generation

#### 3.1 提取相关节点的上下文内容

In [ ]:
# 收集所有选中节点的内容作为上下文
context_parts = []
for nid in selected_node_ids:
    node = node_map[nid]
    if 'text' in node and node['text']:
        context_parts.append(f"【{node['title']}】\n{node['text']}")
    elif 'summary' in node:
        context_parts.append(f"【{node['title']}】\n{node['summary']}")
    else:
        context_parts.append(f"【{node['title']}】\n(无内容)")

if context_parts:
    context = "\n\n---\n\n".join(context_parts)
else:
    context = "未找到相关内容"

# 显示上下文预览
print(f"=== 检索到的上下文 (前800字) ===\n")
print(context[:800])
if len(context) > 800:
    print(f"\n... (共 {len(context)} 字)")

Retrieved Context:

## 5. Conclusion, Limitations, and Future Work

In this work, we share our journey in enhancing model reasoning abilities through reinforcement
learning. DeepSeek-R1-Zero represents a pure RL approach without relying on cold-start data,
achieving strong performance across various tasks. DeepSeek-R1 is more powerful, leveraging cold-
start data alongside iterative RL fine-tuning. Ultimately, DeepSeek-R1 achieves performance
comparable to OpenAI-o1-1217 on a range of tasks.

We further explore distillation the reasoning capability to small dense models. We use DeepSeek-R1
as the teacher model to generate 800K training samples, and fine-tune several small dense models.
The results are promising: DeepSeek-R1-Distill-Qwen-1.5B outperforms GPT-4o and Claude-3.5-Sonnet on
math benchmarks with $28.9 \%$ on AIME and $83.9 \%$ on MATH. Other dense models also achieve
impressive results, significantly outperforming other instructiontuned models based on the same
underlying che

#### 3.2 基于检索到的上下文生成答案

In [ ]:
# 构建答案生成的提示词
answer_prompt = f"""根据以下上下文回答问题。

问题: {query}

上下文: 
{context}

请基于上下文内容，给出清晰、准确的回答。如果上下文中没有相关信息，请说明。
"""

print("=== 正在生成答案... ===\n")
answer = call_llm(answer_prompt)

print("=== 生成的答案 ===\n")
print(answer)

Generated Answer:

The conclusions in this document are:

- DeepSeek-R1-Zero, a pure reinforcement learning (RL) approach without cold-start data, achieves
strong performance across various tasks.
- DeepSeek-R1, which combines cold-start data with iterative RL fine-tuning, is more powerful and
achieves performance comparable to OpenAI-o1-1217 on a range of tasks.
- Distilling DeepSeek-R1’s reasoning capabilities into smaller dense models is promising; for
example, DeepSeek-R1-Distill-Qwen-1.5B outperforms GPT-4o and Claude-3.5-Sonnet on math benchmarks,
and other dense models also show significant improvements over similar instruction-tuned models.

These results demonstrate the effectiveness of the RL-based approach and the potential for
distilling reasoning abilities into smaller models.


---

## 完整流程总结

本 notebook 演示了 **本地化的无向量 RAG** 工作流程:

1. **加载本地结构文件**: 使用 PageIndex 预先生成的文档树结构 (JSON 格式)
2. **树搜索检索**: 通过 LLM 推理，根据节点摘要找出与问题相关的节点
3. **上下文提取**: 从选中节点中提取文本内容作为上下文
4. **答案生成**: 基于检索到的上下文，使用 LLM 生成最终答案

### 与云端 API 版本的区别

| 特性 | 云端 API 版本 | 本地版本 |
|------|--------------|---------|
| 结构生成 | 调用 PageIndex API | 使用本地 `page_index.py` 生成 |
| 数据存储 | 云端托管 | 本地 JSON 文件 |
| LLM 调用 | OpenAI API | 兼容 OpenAI 的任意 API (如阿里云 DashScope) |
| 隐私性 | 数据上传云端 | 数据完全本地化 |

## 下一步

- 修改 `STRUCTURE_PATH` 使用你自己的文档结构文件
- 修改 `query` 测试不同的问题
- 调整 `MODEL` 使用不同的 LLM 模型

---

© 2025 PageIndex 本地化 RAG 示例